In [1]:
import random
import numpy as np

In [2]:
class Maze:
    """Represents the maze environment."""

    def __init__(self):
        """Initialize the maze with the map, dimensions, starting state, terminal states, and actions."""
        self.map = [[-1, -1, -1, 40],
                    [-1, -1, -10, -10],
                    [-1, -1, -1, -1],
                    [10, -2, -1, -1]]
        self.num_rows = len(self.map)
        self.num_cols = len(self.map[0])
        self.start_state = (3, 2)  # row, col
        self.terminal_states = [(0, 3), (3, 0)]
        self.agent_pos = self.start_state
        self.actions = {'↑': (-1, 0), '→': (0, 1), '↓': (1, 0), '←': (0, -1)}

    def step(self, action):
        """Take a step in the environment based on the given action.

        Args:
            action (str): The action to take.

        Returns:
            tuple: The next state, reward, and done flag.
        """
        move = self.actions[action]
        row, col = self.agent_pos
        new_row, new_col = row + move[0], col + move[1]

        # Check if new location is within map boundaries
        if (0 <= new_row < self.num_rows) and (0 <= new_col < self.num_cols):
            self.agent_pos = (new_row, new_col)
            reward = self.map[new_row][new_col]
            done = self.agent_pos in self.terminal_states
        else:
            # If the new location is outside the map, stay in the same spot and get the same punishment again
            reward = self.map[row][col]
            done = False

        return self.agent_pos, reward, done

In [3]:
class Agent:
    """Represents the agent interacting with the maze environment."""

    def __init__(self):
        """Initialize the agent with the maze, policy, and value function."""
        self.maze = Maze()
        self.policy = Policy(self.maze)
        self.value_function = {}

    def act(self, state):
        """Perform an action in the environment based on the current state.

        Args:
            state (tuple): The current state.

        Returns:
            tuple: The next state, reward, and done flag.
        """
        action = self.policy.select_action(state)
        next_state, reward, done = self.maze.step(action)
        return next_state, reward, done

In [4]:
class Policy:
    """Represents the policy for selecting actions."""

    def __init__(self, maze):
        """Initialize the policy with the available actions.

        Args:
            maze (Maze): The maze environment.
        """
        self.actions = {'↑': (-1, 0), '→': (0, 1), '↓': (1, 0), '←': (0, -1)}
        self.maze = maze

    def select_action(self, state):
        """Selects an action to take based on the current state.

        This method randomly selects an action from the available actions defined in the policy.

        Args:
            state (tuple): The current state.

        Returns:
            str: The selected action.
        """
        return random.choice(list(self.actions.keys()))

In [5]:
def value_iteration(agent, delta=0.01, gamma=1.0):
    """
    Perform value iteration to calculate the optimal value function.

    Args:
        agent (Agent): The agent interacting with the environment.
        delta (float): The threshold for convergence.
        gamma (float): The discount factor.
    """

    # Initialize the value function
    for row in range(agent.maze.num_rows):
        for col in range(agent.maze.num_cols):
            agent.value_function[(row, col)] = 0

    k = 0
    while True:
        delta_k = 0
        for row in range(agent.maze.num_rows):
            for col in range(agent.maze.num_cols):
                if (row, col) in agent.maze.terminal_states:
                    continue

                max_value = float('-inf')
                for action in agent.policy.actions.keys():
                    next_row, next_col = row + agent.policy.actions[action][0], col + agent.policy.actions[action][1]
                    if (0 <= next_row < agent.maze.num_rows) and (0 <= next_col < agent.maze.num_cols):
                        next_state = (next_row, next_col)
                        # Calculate the value of the next state using the Bellman equation
                        next_value = agent.maze.map[next_row][next_col] + gamma * agent.value_function[next_state]
                        max_value = max(max_value, next_value)

                delta_k = max(delta_k, abs(max_value - agent.value_function[(row, col)]))
                agent.value_function[(row, col)] = max_value

        k += 1
        print(f'k={k}')
        print(f'delta={delta_k}')
        print_value_function(agent.value_function, agent.policy)
        if delta_k < delta:
            break


def print_value_function(value_function, policy):
    """
    Print the value function and policy.

    Args:
        value_function (dict): The value function.
        policy (Policy): The policy for selecting actions.
    """
    print("Utility:")
    for row in range(policy.maze.num_rows):
        for col in range(policy.maze.num_cols):
            # Print the utility value of each state
            print(f'{value_function[(row, col)]:>4}', end=' ')
        print()

    print("Policy:")
    for row in range(policy.maze.num_rows):
        for col in range(policy.maze.num_cols):
            if (row, col) in policy.maze.terminal_states:
                # Print a circle symbol for terminal states
                print('○', end=' ')
            else:
                state = (row, col)
                max_value = float('-inf')
                best_action = None
                for action in policy.actions.keys():
                    next_row, next_col = row + policy.actions[action][0], col + policy.actions[action][1]
                    if (0 <= next_row < policy.maze.num_rows) and (0 <= next_col < policy.maze.num_cols):
                        next_state = (next_row, next_col)
                        next_value = value_function[next_state]
                        if next_value > max_value:
                            max_value = next_value
                            best_action = action

                # Print the optimal action symbol for each state
                print(best_action, end=' ')
        print()

agent = Agent()
value_iteration(agent, delta=0.01, gamma=1.0)

k=1
delta=40.0
Utility:
-1.0 -1.0 40.0    0 
-1.0 -1.0 39.0 40.0 
10.0  9.0 29.0 30.0 
   0 10.0 28.0 29.0 
Policy:
→ → ↓ ○ 
↓ → ↑ ← 
→ → ↑ ↑ 
○ → ↑ ↑ 
k=2
delta=40.0
Utility:
-2.0 39.0 40.0    0 
 9.0 38.0 39.0 40.0 
10.0 37.0 36.0 35.0 
   0 36.0 35.0 34.0 
Policy:
→ → ↓ ○ 
→ ↑ ↑ ← 
→ ↑ ↑ ↑ 
○ ↑ ↑ ↑ 
k=3
delta=40.0
Utility:
38.0 39.0 40.0    0 
37.0 38.0 39.0 40.0 
36.0 37.0 36.0 35.0 
   0 36.0 35.0 34.0 
Policy:
→ → ↓ ○ 
↑ ↑ ↑ ← 
↑ ↑ ↑ ↑ 
○ ↑ ↑ ↑ 
k=4
delta=0
Utility:
38.0 39.0 40.0    0 
37.0 38.0 39.0 40.0 
36.0 37.0 36.0 35.0 
   0 36.0 35.0 34.0 
Policy:
→ → ↓ ○ 
↑ ↑ ↑ ← 
↑ ↑ ↑ ↑ 
○ ↑ ↑ ↑ 
